# CLASE 4.3: OPTIMIZACIÓN
---
## Introducción.
Cuando colgamos una foto en la pared, a veces es difícil dejarla bien derecha. En general, queremos hacer un determinado ajuste, detenernos, evaluar la horizontalidad de la foto, y volver a repetir el proceso. Dicha secuencia de acciones es un ejemplo (trivial, si nos permitimos la licencia de decirlo así...) de resolución de un problema de optimización. Estamos cambiando la orientación de nuestra foto hasta que satisfaga nuestro requerimiento. Esto es, que su ángulo respecto de la horizontal sea igual a cero.

Un **problema de optimización** puede definirse *a groso modo* como uno que posee *infinitas* soluciones, pero sólo una que podemos denominar, bajo determinados criterios, como la *mejor* solución de todas ellas. El conjunto de infinitas soluciones que podemos asumir, en primera instancia, como *candidatas* para resolver nuestro problema suele denominarse como **espacio de soluciones factibles** del mismo, mientras que la *mejor* solución de dicho espacio, que puede ser estimada por una generosa variedad de métodos tanto algebraicos como numéricos, suele denominarse como **solución óptima** del problema. El *criterio* que permite establecer cúal es la mejor solución en un problema de este tipo guarda relación con los valores que toma una función, denominada como **función objetivo**, sobre el conjunto de soluciones factibles. Normalmente, estamos interesados en el o los valores máximos y mínimos que toma dicha función.

Consideremos, por ejemplo, la función cuadrática $f(x)=(x-3)^{2}$. Queremos encontrar el valor de $x$ tal que $f$ alcanza su valor mínimo. En términos gráficos, al ser $f$ una función cuadrática, sabemos que ésta describe una parábola en $\mathbb{R}^{2}$ con concavidad positiva (apuntando *hacia arriba*). Por lo tanto, es evidente que $f$ tiene un valor mínimo justo en el vértice de dicha parábola, cuando $x=3$.

En general, del curso de cálculo diferencial, sabemos que toda función $f$ que satisfaga el [teorema de Rolle](https://en.wikipedia.org/wiki/Rolle%27s_theorem) será tal que, para todo intervalo abierto $(a, b)$ donde $f$ es diferenciable y $f(a)=f(b)$, siempre existirá un punto interior $c$, tal que $a<c<b$, donde $f'(c)=0$. El punto $x=c$ es llamado **punto crítico** de $f$, y dará lugar a un valor máximo o mínimo de $f$ dependiendo del signo de la derivada $f'(x)$ para puntos a ambos lados de $c$ (criterio de la primera derivada). Si nos referimos a la función cuadrática $f(x)=(x-3)^{2}$, es fácil determinar que $f'(x)=2(x-3)$, de donde sabemos que el único punto crítico de $f$ es $x_{0}=3$. Debido a que la derivada de $f$ es negativa a la izquierda de $x_{0}$ y positiva a la derecha de $x_{0}$, se tendrá que $x_{0}=3$ es un mínimo de $f$.

El ejemplo anterior pone de manifiesto un problema sencillo de optimización univariable que es resoluble de forma analítica, ya que conocemos la función objetivo y no existen restricciones que limiten el espacio de soluciones factibles del problema. Sin embargo, si esta función fuese matemáticamente de mayor complejidad (por ejemplo, si la correspondiente expresión tuviera múltiples términos, varios puntos con derivada nula –máximos y/o mínimos locales, o puntos de ensilladura, que son puntos donde la derivada es nula, pero que no resultan en valores máximos y/o mínimos–, relaciones no lineales, o fuera dependiente de más de una variable), su optimización por métodos manuales sería, como mínimo, una misión extremadamente ardua y muchas veces imposible de resolver en términos algebraicos.

Podemos imaginar a la función objetivo en un problema de optimización como la representación de un paisaje, digamos de un valle, para el cual deseamos encontrar el punto con la mínima elevación posible. Tal analogía pone de manifiesto unq de las principales dificultades al enfrentarnos a un problema de este tipo: Si estuviéramos parados en cualquier valle, con montañas a nuestro alrededor ¿Cómo sabemos si aquel valle es el punto más bajo en toda la extensión de interés? O bien ¿Este valle es el más bajo, siendo que hay otros valles más allá de dichas montañas? En una jerga más formal ¿Cómo sabemos si no nos encontramos en un mínimo local, que corresponde a un valor mínimo de la función objetivo, pero sólo en un entorno local?

## Conceptos preliminares.
La situación presentada previamente pone de manifiesto que un problema de optimización corresponde a un desafío que puede resultar significativamente difícil bajo determinadas condiciones. Para poder entender dicha dificultad, será bueno refrescar algunos conocimientos que, en general, son repasados en detalle en los cursos de cálculo diferencial. No nos extenderemos demasiado aquí en ellos, pero sí será útil tenerlos en consideración. Ante todo, intentaremos dar ejemplos gráficos de lo que estamos hablando, para lo cual haremos un uso intensivo de la librería **Matplotlib**, a fin de graficar algunos resultados. Por supuesto, nos apoyaremos igualmente con **Numpy**, a fin de construir algunos arreglos de datos que nos permitirán describir ciertas funciones de interés:

In [1]:
# Importación de librerías.
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [2]:
from IPython.core.display import HTML
from mpl_toolkits import mplot3d

In [3]:
# Setting de parámetros de graficación.
sns.set()
plt.rcParams["figure.dpi"] = 200 # Resolución de nuestras figuras.
plt.style.use("seaborn-v0_8-white") # Template de estilo.

In [4]:
# Centramos el output de los gráficos en este notebook.
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

En lo que resta de esta subsección, trabajaremos con campos escalares. Esto es, funciones reales o escalares de variable vectorial, del tipo $f:U\subseteq \mathbb{R}^{n} \rightarrow \mathbb{R}$, para $n\geq 2$, donde $U$ es un conjunto abierto de $\mathbb{R}^{n}$. Estamos interesados en resolver, mediante herramientas computacionales, problemas del tipo:

$$\begin{array}{ll}\min_{\mathbf{x} } &f\left( \mathbf{x} \right)  \\ \mathrm{s} .\mathrm{a} .:&g_{i}\left( \mathbf{x} \right)  \leq 0\\ &h_{j}\left( \mathbf{x} \right)  =0\end{array}$$
</p> <p style="text-align: right;">$(3.1)$</p>

Donde $1\leq i\leq m$ y $1\leq j\leq p$, donde:
- $f:U\subseteq \mathbb{R}^{n} \rightarrow \mathbb{R}$ es una **función objetivo** que deseamos minimizar sobre el vector $\mathbf{x}$ de $n$ variables.
- El conjunto $g_{i}(\mathbf{x})=0$ es denominado **restricciones de desigualdad**.
- El conjunto $h_{j}(\mathbf{x})=0$ es denominado **restricciones de igualdad**.
- Y por supuesto, $m,p\geq 0$.

Si $m=p=0$, el problema (3.1), llamado **problema de optimización restringida**, se reduce a uno del tipo **no restringido**. Por convención, la forma estándar anterior define un problema de minimización. Un problema de maximización puede tratarse negando la función objetivo; es decir, escribiendo $\max(f(\mathbf{x}))=-\min(f(\mathbf{x}))$.

Sea $f:U\subseteq \mathbb{R}^{n} \rightarrow \mathbb{R}$ una función definida en el conjunto abierto $U$ de $\mathbb{R}^{n}$. Diremos que $f$ tiene un **mínimo (o un máximo) local o relativo** en el punto $x_{0}\in U$, si $f\left( \mathbf{x}_{0} \right)  \leq f\left( \mathbf{x} \right)$ ($f\left( \mathbf{x}_{0} \right)  \geq f\left( \mathbf{x} \right)$, respectivamente) para todo $\mathbf{x}$ en una bola $B$ de centro en $\mathbf{x}_{0}$. Es decir, al igual que en el caso de las funciones de una variable, la función $f$ tendrá un mínimo local en $\mathbf{x}_{0}\in U$ si $f(\mathbf{x}_{0})$ es el valor más pequeño de todos los valores de $f(\mathbf{x})$ para $\mathbf{x}$ en una bola $B$ de centro en $\mathbf{x}_{0}$.